### 학생 정보 구조화 시스템

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

class StudentProfile(BaseModel):
    name: str = Field(description="학생 이름")
    age: int = Field(description="학생 나이")
    major: str = Field(description="학생 전공")
    hobbies: List[str] = Field(description="학생 취미 리스트")
    goal: str = Field(description="학생의 목표")

parser = PydanticOutputParser(pydantic_object=StudentProfile)

template = """
당신은 텍스트 분석 전문가입니다.
사용자가 입력한 자유 형식의 자기소개 문장에서
아래 항목들을 추출하여 구조화하세요.

텍스트: {intro}

{format_instructions}

중요 규칙:
1. 반드시 JSON 형식만 출력하세요.
2. 다른 설명이나 문장을 출력하지 마세요.
3. 모든 문자열은 큰따옴표(")를 사용하세요.
"""

prompt = ChatPromptTemplate.from_template(template)

prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"intro": intro_text})

print(type(output))
pprint(output)



<class '__main__.StudentProfile'>
StudentProfile(name='김민수', age=22, major='컴퓨터공학', hobbies=['게임하기', '영화보기', '코딩'], goal='훌륭한 개발자가 되는 것')
